In [61]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

print('hello')
# 버전 확인
print(f"pandas 버전: {pd.__version__}")
print(f"numpy 버전: {np.__version__}")


hello
pandas 버전: 2.3.3
numpy 버전: 2.4.0


In [62]:

# 리스트로 Series 생성
sales = pd.Series([120, 150, 180, 200, 160])
print("매출 데이터:")
print(sales)
print(f"\n타입: {type(sales)}")

# NumPy 배열
numpy_array = np.array([10, 20, 30, 40, 50])
print("\nNumPy 배열:", numpy_array)
print("타입:", type(numpy_array))
print("데이터 타입:", numpy_array.dtype)

# Plotly
fig = go.Figure()
fig.show()

매출 데이터:
0    120
1    150
2    180
3    200
4    160
dtype: int64

타입: <class 'pandas.core.series.Series'>

NumPy 배열: [10 20 30 40 50]
타입: <class 'numpy.ndarray'>
데이터 타입: int64


In [63]:
#현재 작업 경로 확인
import os
print(os.getcwd())

#파일 불러오기
df = pd.read_csv('rd/union1.csv', encoding='utf-8')
print("cp949로 로드 성공!")

c:\Users\user\Desktop\S.T.A 새싹\bds\budongsan3_project\ysy
cp949로 로드 성공!


In [71]:
#데이터 살펴보기

df.head()
df.info()
df.describe()
df.shape
df.columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29712 entries, 0 to 29711
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   자치구코드   29712 non-null  int64         
 1   자치구명    29712 non-null  object        
 2   법정동코드   29712 non-null  int64         
 3   법정동명    29712 non-null  object        
 4   본번      29708 non-null  float64       
 5   부번      29708 non-null  float64       
 6   건물명     29712 non-null  object        
 7   계약일     29712 non-null  datetime64[ns]
 8   물건금액    29712 non-null  int64         
 9   건물면적    29712 non-null  float64       
 10  층       29712 non-null  int64         
 11  건축년도    29712 non-null  int64         
 12  건물용도    29712 non-null  object        
 13  신고구분    29712 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(5)
memory usage: 3.2+ MB


['자치구코드',
 '자치구명',
 '법정동코드',
 '법정동명',
 '본번',
 '부번',
 '건물명',
 '계약일',
 '물건금액',
 '건물면적',
 '층',
 '건축년도',
 '건물용도',
 '신고구분']

⭐목표 정의

계약일의 흐름에 따라 부동산 실거래가가 어떻게 변하는지 자치구명, 법정동명 차원에서 분석

In [73]:
#계약일을 datetime형식으로 변환
df['계약일'] = pd.to_datetime(df['계약일'], format= '%Y%m%d')
df['물건금액'] = round(df['물건금액'], 0)

print(df['계약일'].head())
print(df['물건금액'].head())

0   2023-12-31
1   2023-12-31
2   2023-12-30
3   2023-12-30
4   2023-12-30
Name: 계약일, dtype: datetime64[ns]
0    182000
1    162500
2     25700
3    210000
4    323000
Name: 물건금액, dtype: int64


In [66]:

#자치구-법정동 기준
trend_detail = df.groupby(['계약일','자치구명','법정동명'])['물건금액'].mean().round(0).unstack(level=['자치구명','법정동명']).sort_index(axis=1)

#자치구별 기준
trend_gu = trend_detail.groupby(level='자치구명', axis=1).mean().round(0)

#전체 강남3구 기준
trend_total = df.groupby(['계약일'])['물건금액'].mean().round(0)  
overall_avg = trend_total.mean().mean()

print(f"전체 강남3구 기준 평균거래금액: {overall_avg:,.2f}만 원")

print(trend_detail.head())
print(trend_total.head())

전체 강남3구 기준 평균거래금액: 218,501.87만 원
자치구명             강남구                                             ... 송파구      \
법정동명             개포동 논현동       대치동 도곡동 삼성동 세곡동 수서동 신사동 압구정동 역삼동  ... 문정동 방이동   
계약일                                                              ...           
2023-01-01       NaN NaN       NaN NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   
2023-01-02  166900.0 NaN       NaN NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   
2023-01-03  180625.0 NaN  186000.0 NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   
2023-01-04       NaN NaN       NaN NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   
2023-01-05  204000.0 NaN  300000.0 NaN NaN NaN NaN NaN  NaN NaN  ... NaN NaN   

자치구명                                                    
법정동명       삼전동 석촌동 송파동       신천동 오금동       잠실동 장지동 풍납동  
계약일                                                     
2023-01-01 NaN NaN NaN       NaN NaN       NaN NaN NaN  
2023-01-02 NaN NaN NaN       NaN NaN       NaN NaN NaN  
2023-01-03 NaN NaN NaN  172000.0 NaN  217

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2036\2182708038.py:5: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.



In [67]:
# 인덱스를 datetime으로 변경⭐⭐⭐ 
# groupby하면서 '계약일'이 인덱스가 되면서 datetime이 풀렸을 가능성이 큼
trend_detail.index = pd.to_datetime(trend_detail.index)
trend_gu.index = pd.to_datetime(trend_gu.index)
trend_total.index = pd.to_datetime(trend_total.index)

#구-동 일별 데이터 ==> 월별로 resampling하기
trend_detail_monthly = trend_detail.resample('MS').mean()

# 구별 데이터를 일별 ==> 월별로 resampling하기
trend_gu_monthly = trend_gu.resample('MS').mean()

# 강남3구 데이터를 일별 ==> 월별로 resampling하기
trend_total_monthly = trend_total.resample('MS').mean()

print(trend_detail_monthly.head())
print(trend_gu_monthly.head())
print(trend_total_monthly.head())

print(f"시구별 데이터 행 개수: {len(trend_detail_monthly)}")
print(f"구별 데이터 행 개수: {len(trend_gu_monthly)}")
print(f"강남3구 전체 평균 행 개수: {len(trend_total_monthly)}")


자치구명                  강남구                                               \
법정동명                  개포동            논현동            대치동            도곡동   
계약일                                                                      
2023-01-01  185056.050000            NaN  250450.000000  233100.000000   
2023-02-01  164280.913043  126671.428571  212597.368421  182848.571429   
2023-03-01  153964.600000   86516.666667  243302.062500  205575.000000   
2023-04-01  199624.421053  126125.000000  258393.117647  254994.700000   
2023-05-01  184419.350000  179471.428571  246278.652174  265224.150000   

자치구명                                                                         \
법정동명                  삼성동       세곡동            수서동       신사동           압구정동   
계약일                                                                           
2023-01-01   35250.000000       NaN  134714.285714   62500.0  304250.000000   
2023-02-01  176750.000000  137250.0  111549.272727       NaN  365214.285714   
2023-03-01  

In [68]:
# # 데이터가 없는 달(NaN) 제거
# trend_detail_monthly = trend_detail_monthly.dropna()
# trend_gu_monthly = trend_gu_monthly.dropna()
# trend_total_monthly = trend_total_monthly.dropna()

# print(trend_detail_monthly.head())

In [75]:
print(f"전체 행 개수: {len(trend_total)}")

전체 행 개수: 1081


In [ ]:
###시행착오 1:  x축 간격 조정 필요, 각 꺾은선 색상 표시 

# import plotly.graph_objects as go

# # 1. 인덱스 변환 (Period -> Timestamp)
# # Plotly는 '기간'보다 '시점' 데이터를 정확히 인식합니다.
# plot_gu = trend_gu_monthly.copy()
# plot_total = trend_total_monthly.copy()

# if hasattr(plot_gu.index, 'to_timestamp'):
#     plot_gu.index = plot_gu.index.to_timestamp()
# if hasattr(plot_total.index, 'to_timestamp'):
#     plot_total.index = plot_total.index.to_timestamp()

# # 2. 도표 객체 생성
# fig = go.Figure()

# # 3. 구별 월별 추이 선 추가 (연한 색상으로 배경 처리)
# for gu in plot_gu.columns:
#     fig.add_trace(go.Scatter(
#         x=plot_gu.index,
#         y=plot_gu[gu],
#         mode='lines',
#         name=gu,
#         line=dict(width=1),
#         opacity=0.2, # 구별 선은 아주 연하게 해서 겹쳐도 복잡하지 않게 함
#         hovertemplate='%{x|%Y-%m}<br>' + gu + ': %{y:,.0f}만 원<extra></extra>'
#     ))

# # 4. 서울 전체 평균 선 추가 (진한 색상으로 강조)
# fig.add_trace(go.Scatter(
#     x=plot_total.index,
#     y=plot_total.values,
#     mode='lines+markers', # 점을 추가해서 월별 데이터임을 명시
#     name='<b>⭐ 서울 전체 평균</b>',
#     line=dict(color='firebrick', width=4),
#     marker=dict(size=8, symbol='circle'),
#     hovertemplate='<b>%{x|%Y-%m} 평균: %{y:,.0f}만 원</b><extra></extra>'
# ))

# # 5. 레이아웃 및 디자인 설정
# fig.update_layout(
#     title=dict(
#         text='<b>서울시 자치구별 월별 평균 거래가 추이</b>',
#         x=0.5, font=dict(size=22)
#     ),
#     xaxis_title='계약월',
#     yaxis_title='평균 금액 (만 원)',
#     template='plotly_white', # 배경을 하얗고 깔끔하게
#     hovermode='x unified',     # 마우스를 대면 그 달의 모든 구 데이터를 한눈에 보여줌
    
#     xaxis=dict(
#         dtick="M3",            # 3개월 단위로 눈금 표시
#         tickformat="%y년 %m월"  # 축 표시 형식 (23년 01월)
#     ),
#     yaxis=dict(tickformat=',d'), # y축 숫자 콤마 표시
#     legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5)
# )

# fig.show()

In [ ]:
print(f"전체 행 개수: {len(plot_total)}")

In [ ]:
# ###시행착오3: 그래프를 합쳐서 봐야함

# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

# # [핵심] 반드시 '_monthly'가 붙은 변수를 사용해야 합니다!
# # 만약 이 변수가 없다면 위에서 만든 trend_gu_monthly를 여기 대입하세요.
# plot_gu = trend_gu_monthly.copy() 

# # Plotly 인식용 타임스탬프 변환
# if hasattr(plot_gu.index, 'to_timestamp'):
#     plot_gu.index = plot_gu.index.to_timestamp()

# # 4행 1열 서브플롯 생성
# fig = make_subplots(
#     rows=4, cols=1, 
#     subplot_titles=("강남구 월별 가격 추이", "송파구 월별 가격 추이", "서초구 월별 가격 추이", "⭐ 강남 3구 전체 평균"),
#     shared_xaxes=False,
#     vertical_spacing=0.1
# )

# targets = ['강남구', '송파구', '서초구']
# colors = ['royalblue', 'forestgreen', 'darkorange']

# # 꺾은선 그래프 추가
# for i, gu in enumerate(targets):
#     fig.add_trace(
#         go.Scatter(
#             x=plot_gu.index, 
#             y=plot_gu[gu], 
#             mode='lines+markers', # 이제 한 달에 점이 딱 하나씩만 찍힐 겁니다
#             line=dict(color=colors[i], width=2),
#             marker=dict(size=6)
#         ),
#         row=i+1, col=1
#     )

# # 강남 3구 전체 평균 (이미 월별로 계산된 값을 사용)
# if '강남3구_전체' not in plot_gu.columns:
#     plot_gu['강남3구_전체'] = plot_gu[targets].mean(axis=1)

# fig.add_trace(
#     go.Scatter(
#         x=plot_gu.index, 
#         y=plot_gu['강남3구_전체'], 
#         mode='lines+markers',
#         line=dict(color='firebrick', width=3),
#         marker=dict(size=7, symbol='diamond')
#     ),
#     row=4, col=1
# )

# # 디자인 및 X축 설정
# fig.update_layout(height=1200, template='plotly_white', showlegend=False)
# fig.update_xaxes(tickformat="%y년 %m월", dtick="M3", tickangle=45)
# fig.update_yaxes(tickformat=',d')

# print(f"현재 그래프에 사용된 데이터 행 개수: {len(plot_gu)}") # 이 숫자가 30~40 사이여야 합니다!
# fig.show()

In [79]:
import plotly.graph_objects as go

# 1. 데이터 복사 및 타임스탬프 변환
plot_gu = trend_gu_monthly.copy()
if hasattr(plot_gu.index, 'to_timestamp'):
    plot_gu.index = plot_gu.index.to_timestamp()

# 2. 강남 3구 통합 데이터 계산 (이미 되어있다면 생략 가능)
targets = ['강남구', '송파구', '서초구']
if '강남3구_전체' not in plot_gu.columns:
    plot_gu['강남3구_전체'] = plot_gu[targets].mean(axis=1)

# 3. 단일 도표 객체 생성
fig = go.Figure()

# 4. 각 구별 꺾은선 추가 (강남, 송파, 서초)
colors = {'강남구': 'royalblue', '송파구': 'forestgreen', '서초구': 'darkorange'}

for gu in targets:
    fig.add_trace(go.Scatter(
        x=plot_gu.index,
        y=plot_gu[gu],
        mode='lines',
        name=gu,
        line=dict(color=colors[gu], width=2),
        hovertemplate='%{x|%Y-%m}<br>' + gu + ': %{y:,.0f}만 원<extra></extra>'
    ))

# 5. 강남 3구 전체 평균 선 추가 (가장 굵게 강조)
fig.add_trace(go.Scatter(
    x=plot_gu.index,
    y=plot_gu['강남3구_전체'],
    mode='lines',
    name='<b>⭐ 강남 3구 전체 평균</b>',
    line=dict(color='firebrick', width=4), # 주인공은 가장 굵게!
    hovertemplate='<b>%{x|%Y-%m} 통합 평균: %{y:,.0f}만 원</b><extra></extra>'
))

# 6. 전체 디자인 및 X축 날짜 설정
fig.update_layout(
    title=dict(
        text='<b>강남 3구(강남·송파·서초) 및 통합 월별 가격 추이 비교</b>',
        x=0.5, font=dict(size=22)
    ),
    # 1. 그래프 전체의 가로(width)와 세로(height) 크기 설정
    width=1200,         # 기존보다 가로 길이를 충분히 늘림 (기본값은 보통 700~1000)
    height=700,         # 세로 길이는 기존 유지
    
    xaxis=dict(title=dict(text='계약월', standoff=30)),
    yaxis_title='평균 거래가 (만 원)',
    template='plotly_white',
    hovermode='x unified',
    
    # 범례를 하단 가로 방향으로 설정
    legend=dict(
        orientation="h",    # 가로 방향 (Horizontal)
        yanchor="top",      # 범례 상단을 기준으로 위치 지정
        y=-0.15,            # Y축 아래로 충분히 내림 (값이 작을수록 아래로 내려감)
        xanchor="center",   # 범례 중앙을 기준으로
        x=0.5               # 그래프 중앙에 배치
    ),
    ),

# 1. 데이터의 시작과 끝 날짜 파악
start_date = plot_gu.index.min()
end_date = plot_gu.index.max()

# 2. X축 설정 업데이트
fig.update_xaxes(
    range=[start_date, end_date],  # 데이터가 있는 시작일과 종료일로 범위 고정
    tickvals=plot_gu.index,
    ticktext=[f"{m.month}월" for m in plot_gu.index], # '01월' 대신 '1월'로 변환
    tickformat="%m월", 
    dtick="M1", 
    tickangle=0,
    autorange=False                # 자동 범위 설정을 끄고 지정한 range를 따름
)

# 2. 연도 구분선 추가 (24년, 25년 시작 지점)
# 흐린 회색 점선으로 설정하여 배경처럼 보이게 합니다.
years = [2024, 2025, 2026]
for year in years:
    # 해당 연도 1월 1일 위치에 선 추가
    year_start = f"{year}-01-01"
    
    # 데이터 범위 내에 있는 연도만 선을 긋습니다.
    if plot_gu.index.min() <= pd.to_datetime(year_start) <= plot_gu.index.max():
        fig.add_vline(
            x=year_start, 
            line_width=1.5, 
            line_dash="solid",              # 'dash'에서 'solid'(실선)로 변경
            line_color="rgba(220, 220, 220, 0.5)" # 더 흐린 회색 (숫자가 높을수록 밝음)
        )
        
        # 선 위에 '24년', '25년' 텍스트 추가 (선택 사항)
        fig.add_annotation(
            x=year_start, y=1, yref="paper",
            text=f"<b>{year}년</b>",
            showarrow=False, font=dict(color="gray", size=12),
            xanchor="center", yanchor="bottom"
        )

# Y축 숫자 콤마 설정
fig.update_yaxes(tickformat=',d')

fig.show()

## 왜 송파구만 저렇게 거래가가 낮지?

### 송파구 vs (강남구, 서초구) 비교해보자! 

In [83]:
# # 1. 대상 구(강남, 송파, 서초) 필터링
# targets = ['강남구', '송파구', '서초구']
# gu_comparison = df[df['자치구명'].isin(targets)].copy()

# 2. 구별 거래 건수(Count), 평균 금액(Mean), 합계 금액(Sum) 비교
# MECE하게 분석하기 위해 합계까지 포함하여 비교합니다.
comparison_stats = df.groupby('자치구명').agg(
    거래건수=('물건금액', 'count'),      # 얼마나 자주 거래되었나 (Volume)
    평균가=('물건금액', 'mean'),        # 전형적인 가격 수준 (Average)
    합계금액=('물건금액', 'sum')         # 해당 지역 전체 자금 규모 (Total Amount)
).round(0)

# 3. 거래 건수 순으로 정렬하여 보기
print("강남 3구 거래 규모 및 가격 비교")
print(comparison_stats.sort_values(by='거래건수', ascending=False))

강남 3구 거래 규모 및 가격 비교
       거래건수       평균가        합계금액
자치구명                             
송파구   12306  173686.0  2137376727
강남구   10075  256579.0  2585036895
서초구    7331  260071.0  1906578545


### 송파 vs (강남, 서초) 비교 데이터 시각화 

In [ ]:
import plotly.express as px  # 시각화 라이브러리인 Plotly Express를 px라는 별칭으로 가져옵니다.

# 1. 인덱스(자치구명)를 컬럼으로 변환 (시각화 필수 단계)
plot_stats = comparison_stats.reset_index()  # 행 제목(Index)으로 있던 '자치구명'을 일반 데이터 열(Column)로 내려 시각화 도구가 인식하게 합니다.
plot_stats  # 변환된 데이터프레임의 내용을 화면에 출력하여 확인합니다.

# 2. 거래건수 시각화 (시장 활성도 확인)
fig_count = px.bar(  # 막대 그래프(Bar Chart) 객체를 생성하여 fig_count 변수에 저장합니다.
    plot_stats,      # 그래프를 그릴 원천 데이터로 plot_stats를 지정합니다.
    x='자치구명',     # 가로축(X축)에 '자치구명' 데이터를 배치합니다.
    y='거래건수',     # 세로축(Y축)에 '거래건수' 수치를 반영하여 막대 높이를 결정합니다.
    text_auto=True,  # 막대 상단에 실제 거래건수 숫자를 자동으로 표시합니다.
    title='<b>강남 3구 지역별 거래건수 (Volume)</b>',  # 그래프 제목을 굵게 설정합니다.
    color='자치구명', # 각 자치구마다 서로 다른 색상을 부여하여 시각적으로 구분합니다.
    template='plotly_white'  # 배경을 흰색으로 설정하여 차트를 깔끔하고 전문적으로 보이게 합니다.
)
fig_count.update_layout(showlegend=False)  # 이미 X축에 이름이 있으므로 중복되는 범례(Legend)를 숨깁니다.
fig_count.show()  # 완성된 거래건수 그래프를 화면에 띄웁니다.

# 3. 평균가 시각화 (가격 수준 확인)
fig_mean = px.bar(   # 평균 거래가 비교를 위한 새로운 막대 그래프를 생성합니다.
    plot_stats,      # 동일한 요약 데이터를 사용합니다.
    x='자치구명',     # 기준축은 동일하게 '자치구명'으로 설정합니다.
    y='평균가',       # 이번에는 높이 기준을 '평균가' 수치로 바꿉니다.
    text_auto=',.0f', # 숫자에 천 단위 콤마(,)를 넣고 소수점은 없애서 가독성을 높입니다.
    title='<b>강남 3구 지역별 평균 거래가 (Average Price)</b>',  # 가격 비교용 제목을 답니다.
    color='자치구명', # 앞선 그래프와 동일한 색상 체계를 유지하여 통일감을 줍니다.
    template='plotly_white'  # 차트의 스타일을 깨끗한 흰색 배경으로 통일합니다.
)
fig_mean.update_layout(yaxis_title="평균가 (만 원)", showlegend=False) # 세로축 단위를 명시하고 범례를 숨깁니다.
fig_mean.show()  # 완성된 평균가 그래프를 화면에 띄웁니다.

#### 아! 거래총액이 2위인걸로 봐서 평균가격이 낮은 이유는 거래 건수는 많은데 거래가가 낮아서 그렇구나! 
####그럼 이 거래가가 왜 낮지? 혹시 평형수가 작은 아파트들이 강남, 서초에 비해 많은가? 

In [81]:
# 1. 현재 df가 가진 모든 컬럼명을 한 줄로 출력합니다.
print("실제 컬럼명 리스트:", df.columns.tolist()) # 이 출력 결과에서 면적 관련 이름을 찾아야 합니다.

#2. 건물면적을 '평수'로 계산한 컬럼 생성
df['평수'] = df['건물면적'] / 3.3058 
print(df['평수'].head().round(2))

# 3. 평형대 구간(Bin) 나누기
# 0~10평, 10~20평, 20~30평, 30~40평, 40평 이상으로 구분합니다.
bins = [0, 10, 20, 30, 40, 1000] # 구간의 경계값을 설정합니다. # (1000은 40평 이상의 모든 값 포함)
labels = ['10평 이하', '10평대', '20평대', '30평대', '40평 이상'] # 각 구간에 붙일 이름표입니다.

# 4. 평형구분 컬럼 생성
df['평형구분'] = pd.cut(df['평수'], bins=bins, labels=labels, right=False) # 평수 데이터를 기준에 따라 구간별로 분류합니다. # (cut 함수 사용)

# 5. 강남 3구 필터링 및 구별/평형별 거래건수 집계
targets = ['강남구', '송파구', '서초구'] # 비교할 대상 구를 리스트로 만듭니다.
comparison_df = df[df['자치구명'].isin(targets)].copy() # 대상 구의 데이터만 복사해옵니다.

# 6. 거래건수 확인 (물건금액 컬럼의 개수를 카운트)
# 자치구명과 평형구분을 기준으로 묶어 건수를 계산합니다.
result = comparison_df.groupby(['자치구명', '평형구분']).agg(
    거래건수=('물건금액', 'count') # 각 그룹별 물건금액의 개수를 세어 거래건수를 구합니다. # (Aggregation)
).reset_index() # 결과표를 보기 좋게 평면 데이터로 펼칩니다.

# 7. 결과 출력
print("[강남 3구 평형대별 거래건수 비교]")
print(result.pivot(index='평형구분', columns='자치구명', values='거래건수')) # 가로축을 구, 세로축을 평형으로 하는 표로 변환합니다.

실제 컬럼명 리스트: ['자치구코드', '자치구명', '법정동코드', '법정동명', '본번', '부번', '건물명', '계약일', '물건금액', '건물면적', '층', '건축년도', '건물용도', '신고구분']
0    18.01
1    18.15
2     7.83
3    32.43
4    25.71
Name: 평수, dtype: float64
[강남 3구 평형대별 거래건수 비교]
자치구명     강남구   서초구   송파구
평형구분                    
10평 이하   227   144   324
10평대    3132  1594  3556
20평대    3864  3357  5911
30평대    1604  1283  1683
40평 이상  1248   953   832


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2036\3221321972.py:22: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [86]:
import plotly.express as px # 시각화 라이브러리인 Plotly Express를 가져옵니다.

# 1. 시각화를 위해 피벗된 데이터를 다시 길게(Long format) 펼칩니다.
# px.bar를 쓰기 위해서는 인덱스를 컬럼으로 돌려놓아야 합니다.
plot_df = pivot_df.reset_index().melt(id_vars='평형구분', var_name='자치구명', value_name='거래건수') # 요약된 표를 그래프용 데이터로 변환합니다.

# 2. 누적 막대 그래프 생성 (Stacked Bar Chart)
fig = px.bar(
    plot_df, # 그래프를 그릴 데이터를 지정합니다.
    x='자치구명', # 가로축에 구 이름을 배치하여 구별 전체 규모를 비교합니다.
    y='거래건수', # 세로축에 거래건수를 쌓아 올립니다.
    color='평형구분', # 평형대별로 색상을 나누어 막대 안의 비중을 보여줍니다.
    text_auto=True, # 막대 칸 안에 실제 거래건수 숫자를 표시합니다.
    title='<b>강남 3구 평형대별 거래 비중 및 규모 비교</b>', # 그래프 제목을 설정합니다.
    # 색상을 직접 지정하여 직관성을 높일 수 있습니다. (연한 색 -> 진한 색 순서 권장)
    color_discrete_sequence=px.colors.sequential.Viridis, # 조화로운 색상 체계를 적용합니다.
    template='plotly_white' # 배경을 깨끗한 흰색으로 설정합니다.
)

# 3. 그래프 레이아웃 세부 조정
fig.update_layout(
    xaxis_title='자치구명', # X축 제목을 설정합니다.
    yaxis_title='총 거래건수 (건)', # Y축 제목을 설정합니다.
    legend_title='평형대 구분', # 범례 제목을 설정합니다.
    barmode='stack', # 막대를 위로 쌓아 올리는 '누적' 모드를 설정합니다.
    font=dict(size=10) # 글자 크기를 키워 가독성을 높입니다.
)

fig.show() # 완성된 그래프를 화면에 출력합니다.

In [93]:
import plotly.express as px  # 시각화 라이브러리를 가져옵니다.

# 1. 10평 이하 데이터 제거 (Filtering)
plot_df_filtered = plot_df[plot_df['평형구분'] != '10평 이하'].copy() # '10평 이하'를 제외한 데이터만 추출합니다.

# 2. 각 자치구 내에서의 비율(%) 계산
total_counts = plot_df_filtered.groupby('자치구명')['거래건수'].transform('sum') # 각 구별 총합을 구합니다.
plot_df_filtered['비율'] = (plot_df_filtered['거래건수'] / total_counts * 100).round(1) # 백분율을 계산합니다.

# 3. 그래프에 표시할 텍스트 라벨 생성 (예: 60%(3864건))
plot_df_filtered['라벨'] = plot_df_filtered.apply(
    lambda row: f"{row['비율']}%<br>({int(row['거래건수']):,}건)", axis=1 # 라벨 형식을 지정합니다.
)

# 4. 누적 막대 그래프 생성 및 색상 변경
fig = px.bar(
    plot_df_filtered, # 필터링된 데이터를 사용합니다.
    x='자치구명', # 가로축입니다.
    y='거래건수', # 세로축 높이는 거래건수 기준입니다.
    color='평형구분', # 색상 구분 기준입니다.
    text='라벨', # 라벨을 막대 위에 표시합니다.
    title='<b>강남 3구 평형대별 거래 비중 및 규모 비교 (10평 이하 제외)</b>', # 제목 설정입니다.
    # 색상 팔레트 변경: 연한 파랑(낮은 평형)에서 진한 파랑(높은 평형)으로 변경
    color_discrete_sequence=px.colors.sequential.Blues_r, # Blues_r은 색상이 역순으로 적용되어, # (작은 평수 -> 연한 색, 큰 평수 -> 진한 색)
                                                         # 낮은 평수부터 진한 파란색이 되도록 합니다. 
                                                         # 만약 연한 파랑부터 시작하려면 Blues를 사용하세요.
    template='plotly_white' # 배경을 흰색으로 설정합니다.
)

# 5. 상세 레이아웃 업데이트
fig.update_layout(
    title=dict(font=dict(size=28), x=0.5), # 제목 크기 및 위치 조정입니다.
    width=800, # 그래프 너비를 슬림하게 조정합니다.
    height=900, # 그래프 높이를 키워 시인성을 확보합니다.
    xaxis=dict(title="", tickfont=dict(size=16)), # X축 제목 제거 및 글자 크기 조정입니다.
    yaxis=dict(title="총 거래건수 (건)", tickformat=",d", tickfont=dict(size=14)), # Y축 숫자 콤마 표시입니다.
    legend_title=dict(text="평형대 구분", font=dict(size=15)), # 범례 설정입니다.
    bargap=0.4 # 막대 사이의 간격을 벌립니다.
)

# 막대 내부 텍스트 스타일 조정
fig.update_traces(textposition='inside', textfont_size=12, textfont_color='white') # 텍스트 색상을 검정색으로 지정하여 가독성을 높입니다.

fig.show() # 최종 결과물을 출력합니다.